In [8]:
import os
from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

load_dotenv()

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

# langchain支持deepseek以model identifier string方式声明（需要在.env中配置DEEPSEEK_API_KEY），考虑到调用成本，改用免费的model
# agent = create_agent(
#     model = "deepseek-chat",
#     tools = [get_weather],
#     system_prompt = "You are a helpful assistant",
# )

model = ChatOpenAI(
    model="qwen-max",
    temperature=0.1,
    max_tokens=1000,
    timeout=30,
    api_key=os.getenv('API_KEY'),
    base_url=os.getenv('BASE_URL')
)
agent = create_agent(
    model,
    tools = [get_weather],
    system_prompt = "You are a helpful assistant",
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='8702aa1a-b33d-4b93-a282-25773ffb639e'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 240, 'total_tokens': 257, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'qwen-max', 'system_fingerprint': None, 'id': 'chatcmpl-9e123cb4-d7c6-46b1-b829-3f80a6ea0736', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--d2010f4f-8565-4948-a47d-d074d43e4faa-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': 'call_4bb6119b0024408eab1cb7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 240, 'output_tokens': 17, 'total_tokens': 257, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}),
  ToolMessage(content="It's always sunny in sf!", name='get_weather'